In [ ]:
from dotenv import load_dotenv
import os

import xesmf as xe
import xarray as xr
import dask.array as da

from xclim.testing import open_dataset
from xclim.indices.stats import standardized_index_fit_params
from xclim.indices import standardized_precipitation_index



from dask.distributed import Client
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import pandas as pd

from distributed import Client

In [ ]:
load_dotenv()

data_path=os.getenv("data_path")

## Steps in the Code

1. **Environment Setup:**
   - Loads environment variables, including the data path, using dotenv.
   
2. **Data Retrieval and Preprocessing:**
   - Loads CHIRPS dataset from the specified data path.
   - Defines latitude and longitude bounds for the region of interest.
   - Subsets the dataset based on the defined bounds and saves the subset as `ea-chirps-v2.0.monthly.nc`.
   - Renames longitude and latitude dimensions to `lon` and `lat` respectively.
   - Defines a new grid for regridding CHIRPS data to a regular grid with specified latitudes and longitudes.
   - Performs bilinear regridding of CHIRPS data to the new grid and saves the regridded dataset as `ea-km25-chirps-v2.0.monthly.nc`.
   
3. **Dask Client Setup:**
   - Initializes a Dask distributed client for parallel computation with specified number of workers, threads per worker, and memory limit.
   
4. **Standardized Precipitation Index (SPI) Calculation (3-month window):**
   - Loads the regridded CHIRPS dataset.
   - Sets units attribute for precipitation data.
   - Chunks the dataset along the time dimension.
   - Calculates SPI with a 3-month window using xclim library.
   - Computes SPI values.
   - Saves SPI values as a NetCDF file named `ea_chrips_spi3_xclim_20240306.nc`.
   
5. **Standardized Precipitation Index (SPI) Calculation (4-month window):**
   - Loads the regridded CHIRPS dataset.
   - Sets units attribute for precipitation data.
   - Chunks the dataset along the time dimension.
   - Calculates SPI with a 4-month window using xclim library.
   - Computes SPI values.
   - Saves SPI values as a NetCDF file named `ea_chrips_spi4_xclim_20240306.nc`.
   
6. **Client Shutdown:**
   - Closes the Dask client.


In [ ]:
#wget https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_monthly/netcdf/chirps-v2.0.monthly.nc

In [ ]:
# Define the bounds of your region of interest
ds=xr.open_dataset(f'{data_path}chirps-v2.0.monthly.nc')

lat_min, lat_max = 20, 55  # Example latitude bounds
lon_min, lon_max = -15, 25  # Example longitude bounds

# Subset the dataset for the region of interest
subset_ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
subset_ds.to_netcdf(f'{data_path}ea-chirps-v2.0.monthly.nc')

In [ ]:
ds=xr.open_dataset(f'{data_path}ea-chirps-v2.0.monthly.nc')
ds1=ds.rename({'longitude':'lon','latitude':'lat'})
dr = ds1["precip"] 
ds_out = xr.Dataset(
      {"lat": (["lat"], np.arange(-15.0, 25.0, 0.25), {"units": "degrees_north"}),
      "lon": (["lon"], np.arange(20, 55, 0.25), {"units": "degrees_east"}),})
regridder = xe.Regridder(ds1, ds_out, "bilinear")
regridder  # print basic regridder information.
dr_out = regridder(dr, keep_attrs=True)
ds2=dr_out.to_dataset()
ds2.to_netcdf(f'{data_path}ea-km25-chirps-v2.0.monthly.nc')

In [ ]:
# Depending on your workstation specifications, you may need to adjust these values.
# On a single machine, n_workers=1 is usually better.
client = Client(n_workers=3, threads_per_worker=4, memory_limit="2GB")
client

## SPI3

In [ ]:
ch_ds=xr.open_dataset(f'{data_path}ea-km25-chirps-v2.0.monthly.nc')

ch_ds['precip'].attrs['units'] = 'mm/month'
ch_ds1 = ch_ds.chunk(-1)
aa=ch_ds1.precip
spi_3 = standardized_precipitation_index(
     aa,
     freq="MS",
     window=3,
     dist="gamma",
     method="APP",
     cal_start='1991-01-01',
     cal_end='2018-01-01',
)

a_s3=spi_3.compute()

In [ ]:
ch_spi= a_s3.to_dataset(name='spi3')
ch_spi.spi3.nbytes / (1024*1024)
#input_path
ch_spi.to_netcdf(f'{data_path}ea_chrips_spi3_xclim_20240306.nc')

## SPI4

In [ ]:
ch_ds['precip'].attrs['units'] = 'mm/month'
ch_ds1 = ch_ds.chunk(-1)
aa=ch_ds1.precip
spi_4 = standardized_precipitation_index(
     aa,
     freq="MS",
     window=4,
     dist="gamma",
     method="APP",
     cal_start='1991-01-01',
     cal_end='2018-01-01',
)

a_s4=spi_4.compute()

ch_spi4= a_s4.to_dataset(name='spi4')
ch_spi4.spi4.nbytes / (1024*1024)
#input_path
ch_spi4.to_netcdf(f'{data_path}ea_chrips_spi4_xclim_20240306.nc')

In [ ]:
client.close()